In [ ]:
#########################################
#Latest version for S14-experiment
#########################################

#Installation
init_load=False
if init_load:
  !pip install sklearn-relief==1.0.0b2
  !pip install skrebate
  !pip install sklearn-genetic
  !pip install git+https://github.com/smazzanti/mrmr
  !pip install shap
  !pip install Probatus

# Basics
import numpy as np
import pandas as pd
from pathlib import Path
from google.colab import files
from google.colab import drive

#Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from operator import itemgetter
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import RandomizedSearchCV
import lightgbm
#import shap

#GPU
import tensorflow as tf
#tf.test.gpu_device_name()

#Feature selection methods
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
import sklearn_relief as relief
from skrebate import ReliefF
#from mrmr import mrmr_classif
from probatus.feature_elimination import ShapRFECV
from sklearn.feature_selection import RFE # Recursive feature elimination
from sklearn.feature_selection import RFECV # with crossvalidation
from sklearn.svm import SVC
from genetic_selection import GeneticSelectionCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel

#Classification methods
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

#Evaluation
import time
from datetime import datetime, timedelta
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score


#Visualization
%matplotlib inline
from IPython.display import HTML, display
import seaborn as sns
import matplotlib.pyplot as plt

#OS
#Path.home()
#uploaded = files.upload()

#print all rows
pd.set_option('display.max_rows', 20)

#Grouped result
  #grouped_res = exp_result.groupby(['model', 'FS_method', 'Normalization']).size()

def LoadGDriveFile (project, filename):
  drive.mount('/content/drive')
  file_path='/content/drive/MyDrive/PHD/'+project+'/dataset/'+filename
  !cp "/content/drive/MyDrive/PHD/S14/dataset/S14_TOTAL_DATASET_V2.csv" "S14_TOTAL_DATASET_V2.csv"
  #!cp file_path {filename}

#Mount Google drive
drive.mount('/content/drive')

def print_border(type, length=70):
  border = ''
  for i in range(length+1):
    border=border+type
  print (border)

def make_csv_df(dataset,y, sep, imp_mean, split, ratio, featuregroup, dataset_type):
    X_train_pd=[]
    X_test_pd=[]
    y_train_pd=[]
    y_test_pd=[]
    #OrgConcept=[51131148, 51126540,51136012]
    df = pd.read_csv(dataset,sep=sep, low_memory = False)
    rows =len(df)
    print ('Selected dataset type and feature group:', dataset_type,'and' ,featuregroup)
    df=df.head(rows)
    if dataset_type=='RAW':
      if ('PK' in df.columns):
        df=df.drop(columns=['PK'])
      if (featuregroup in (['ORG','IDEATION' ,'FE-GEN'])):
        df=df[df['CREATION_PHASE']==featuregroup]
        print(featuregroup, 'chosen')
      if (featuregroup=='ORGIDEA'):
        print('ORGIDEA chosen')
        df=df[df['CREATION_PHASE'].isin(['ORG','IDEATION'])]
      if (featuregroup=='ORGFE'):
        print('ORGFE chosen')
        df=df[df['CREATION_PHASE'].isin(['ORG','FE-GEN'])]
      display(df.groupby(['CREATION_PHASE', 'CONCEPT_ID'])['PID'].count() )
      print('Unique F-group and unique patients:')
      display(df[['CREATION_PHASE', 'PID']].drop_duplicates().groupby(['CREATION_PHASE'])['PID'].count() )
      df=df.drop(columns=['CREATION_PHASE'])
      df=df.drop(columns=['CONCEPT_ID'])
    if dataset_type=='TIDY':
      print ('chosen TIDY')
      if ('PK' in df.columns):
        df=df.drop(columns=['PK'])
      if (featuregroup=='ORG'):
        df=df[org_list+y]
        print('ORG chosen')
      if (featuregroup=='IDEATION'):
        df=df[idea_list+y]
        print('IDEATION chosen')
      if (featuregroup=='FE-GEN'):
        df=df.drop(columns=org_list+idea_list)
        print('FE-GEN chosen')
      if (featuregroup=='ORGIDEA'):
        df= df[org_list+idea_list+y]
        print('ORGIDEA chosen')
      if (featuregroup=='ORGFE'):
        df=df.drop(columns=idea_list)
        print('ORGFE chosen')
    y_pd=df[y[0]]
    X_pd=df.drop(columns=[y[0]])
    #Replacing NaN with meanvalues
    if (imp_mean):
      X_pd=X_pd.fillna(X_pd.mean())
      y_pd=y_pd.fillna(y_pd.mean())
    #Drop all NaN /zero columns
    X_pd.dropna(how='all', axis=1, inplace=True)
    #Summary of dataset
    X_var=X_pd.columns.values
    print ("Imported number of rows: ", rows,"of",rows, "from dataset:", dataset)
    print ('The dataset has based on featuregroup and dataset type selected:',len(X_var), 'describing features and',len(y), 'dependent variabel:')
    print ('The describing features are: \n',X_var)
    print ('The dependent variable is:',y[0])
    #Splitting into train and test dataset
    if (split):
      X_train, X_test, y_train_list, y_test_list = train_test_split(X_pd, y_pd, test_size=ratio, random_state=8675309)
    y_test= pd.DataFrame(y_test_list)
    y_train= pd.DataFrame(y_train_list)
    print ('Sixe of X_PD end create dataset', X_pd.shape)
    return (X_pd, y_pd, X_train, X_test, y_train, y_test)


def dfNorm(df, type):
    scaler = preprocessing.MinMaxScaler()
    columns=sorted(df)
    df=pd.DataFrame(scaler.fit_transform(pd.DataFrame(df)))
    df.columns=columns
    return (df)

def var_typing (df):
    i = 0
    df['f_code']='-1'
    for i in range (len(df)) :
      feature=df['feature'].iloc[i]
      if (feature[0:1] =='X'):
        feature_code = 'ORG'
      else:
       feature_code = 'FE'
      df['f_code'].iloc[i]=feature_code
      i+=1

def actual_time():
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  return current_time

def time_diff(starttime, endtime):
  if starttime == 0:
    tmdelta=actual_time()
  else:
    period=round(endtime-starttime)
    tmdelta= str(timedelta(seconds = period))
  return tmdelta

def make_PID_unique_df(result_df):
  result_UnPID_df=result_df[['PID','Y1', 'Y_mv']].drop_duplicates()
  print ('Shape of unique PID dataset:',result_UnPID_df.shape, 'versus input', result_df.shape )
  return result_UnPID_df

def add_y_major_vote(result_UnPID_df, group_col, calc_col):
  Y_hat_mean=result_UnPID_df.groupby('PID')['Y_hat'].mean()
  Y_hat_mean=Y_hat_mean.rename("Y_hat_mean")
  Y_hat_mean=Y_hat_mean.to_frame()
  y_pred_hat_mean=pd.merge(result_UnPID_df, Y_hat_mean, how='inner', on='PID')
  y_pred_mv=y_pred_hat_mean['Y_hat_mean'].round()
  result_UnPID_df=y_pred_hat_mean.assign(Y_mv=y_pred_mv)
  if reporting:
    print ('Rows with means >0 AND < 1',Y_hat_mean.loc[lambda x : (x < 1) & (x > 0)] )
    print (y_pred_hat_mean.head(10))
    print ('Rows with Y_hat mean > 0 AND <1', result_UnPID_df.loc[(result_UnPID_df['Y_hat_mean']>0) & (result_UnPID_df['Y_hat_mean']<1)].drop_duplicates() )
  return result_UnPID_df

def FSVarianceThreshold(X, FS_X_train, FS_X_test,y_pd,n):
    threshold = 0
    X_VT_input = X.select_dtypes(include='number')
    FS_VT=VarianceThreshold(threshold=threshold)
    X_VT=FS_VT.fit_transform(X_VT_input)
    print ('Size of Variance threshold ds before masking and type:',X_VT.shape)
    #display('FS_VT before and size, type:',FS_VT, np.size(FS_VT), type(FS_VT))
    X_VT_variance=FS_VT.variances_
    #display('List of variances: ',X_VT_variance)
    drop_columns=X_VT_variance.argsort()[:-n]
    sorted_var=np.sort(FS_VT.variances_)
    sorted_var=sorted_var[sorted_var.argsort()[-n:]]
    X_VT_mask=FS_VT.get_support()

    for i in range(len(drop_columns)):
      index=drop_columns[i]
      X_VT_mask[index] = False

    X_VT_df = X.loc[:, X_VT_mask]
    X_VT_train_df = FS_X_train.loc[:, X_VT_mask]
    X_VT_test_df = FS_X_test.loc[:, X_VT_mask]
    return X_VT_df, X_VT_train_df, X_VT_test_df

def KBestN(X,X_train, X_test,y,n):
    FS_KBestChi2=SelectKBest(chi2, k=n)
    X_KBChi2=FS_KBestChi2.fit_transform(X, y)
    X_KBestChi2_mask=FS_KBestChi2.get_support()
    X_KBChi2_df = X.loc[:, X_KBestChi2_mask]
    X_KBChi2_train_df = X_train.loc[:,X_KBestChi2_mask]
    X_KBChi2_test_df = X_test.loc[:,X_KBestChi2_mask]
    print ('Dataframe with K Best Neighbours (Chi2) for n=',n,'best features')
    return X_KBChi2_df, X_KBChi2_train_df, X_KBChi2_test_df

def KBestMutual(FS_X,FS_X_train, FS_X_test,y,n):
    print ('KBEstMut-func started')
    FS_KBMutual=SelectKBest(mutual_info_classif, k=n)
    X_KBMutual=FS_KBMutual.fit_transform(FS_X, y)
    X_KBMutual_mask=FS_KBMutual.get_support()
    X_KBMutual_df = FS_X.loc[:, X_KBMutual_mask]
    X_KBMutual_train_df = FS_X_train.loc[:, X_KBMutual_mask]
    X_KBMutual_test_df = FS_X_test.loc[:, X_KBMutual_mask]
    print ('Dataframe with K Best Neighbours (Mutual info) for n=',n,'best features')
    return X_KBMutual_df, X_KBMutual_train_df, X_KBMutual_test_df

def Relief (FS_X,FS_X_train, FS_X_test,y,n):
    y=np.reshape(y,len(y)) #change size from (x,1) to (x,)
    FS_Relief=relief.Relief(n_features=n)
    X_Relief=FS_Relief.fit_transform(FS_X, y)
    X_Relief_mask=FS_Relief.get_support()
    X_Relief_df = FS_X.loc[:, X_Relief_mask]
    X_Relief_train_df = FS_X_train.loc[:, X_Relief_mask]
    X_Relief_test_df = FS_X_test.loc[:, X_Relief_mask]
    print ('Dataframe with Relief for n=',n,'best features')
    return X_Relief_df, X_Relief_train_df, X_Relief_test_df

def ReliefSB (FS_X,FS_X_train, FS_X_test,y,n):
    y=np.reshape(y,len(y)) #change size from (x,1) to (x,)
    FS_ReliefSB=ReliefF(n_features_to_select=n)
    X_ReliefSB=FS_ReliefSB.fit_transform(FS_X, y)
    X_ReliefSB_mask=FS_ReliefSB.get_support()
    X_ReliefSB_df = FS_X.loc[:, X_ReliefSB_mask]
    X_ReliefSB_train_df = FS_X_train.loc[:, X_ReliefSB_mask]
    X_ReliefSB_test_df = FS_X_test.loc[:, X_ReliefSB_mask]
    print ('Dataframe with ReliefSB for n=',n,'best features')
    return X_ReliefSB_df, X_ReliefSB_train_df, X_ReliefSB_test_df

def mRMR(FS_X,X_train, X_test,y,n):
    FS_mRMR=mrmr_classif(K=n)
    X_mRMR=FS_mRMR.fit_transform(FS_X,y)
    X_mRMR_mask=FS_mRMR.get_support()
    X_mRMR_df = FS_X.loc[:, X_mRMR_mask]
    X_mRMR_train_df = FS_X.loc[:, X_mRMR_mask]
    X_mRMR_test_df = FS_X.loc[:, X_mRMR_mask]
    print ('Dataframe with mRMR for n=',n,'best features')
    return X_mRMR_df

def GenSelectnew(FS_X,FS_X_train, FS_X_test,y,n):
    estimator = DecisionTreeClassifier()
    iterations=100
    FS_GenSelect=GeneticSelectionCV(estimator, cv=5, verbose=0, scoring="accuracy", max_features=n,
    n_population=100, crossover_proba=0.5, mutation_proba=0.2, n_generations=50, crossover_independent_proba=0.5,
    mutation_independent_proba=0.04, tournament_size=3, n_gen_no_change=10, caching=True, n_jobs=-1)
    X_GenSelect=FS_GenSelect.fit_transform(FS_X, y)
    print(sum(FS_GenSelect.support_))
    print(FS_GenSelect.support_)
    print(FS_GenSelect.generation_scores_)
    print(FS_GenSelect.generation_scores_)
    print (dir(FS_GenSelect))
    X_GenSelect_mask=FS_GenSelect.get_support()

    X_GenSelect_df = FS_X.loc[:, X_GenSelect_mask]
    X_GenSelect_train_df = FS_X_train.loc[:, X_GenSelect_mask]
    X_GenSelect_test_df = FS_X_test.loc[:, X_GenSelect_mask]
    print ('Dataframe with genetic selection for n=',n,'best features')
    return X_GenSelect_df, X_GenSelect_train_df, X_GenSelect_test_df

def RFE_FS(FS_X,FS_X_train, FS_X_test,y,n):
    estimator =DecisionTreeClassifier()
    FS_RFE = RFE(estimator=estimator, n_features_to_select=n, step=1, verbose=0)
    X_RFE=FS_RFE.fit(FS_X, y)
    X_RFE_mask=FS_RFE.get_support()
    X_RFE_df = FS_X.loc[:, X_RFE_mask]
    X_RFE_train_df = FS_X_train.loc[:, X_RFE_mask]
    X_RFE_test_df = FS_X_test.loc[:, X_RFE_mask]
    print ('Dataframe with Recursive feature elimination (RFE) for n=',n,'best features')
    return X_RFE_df, X_RFE_train_df, X_RFE_test_df

def RFECV_FS(FS_X,FS_X_train, FS_X_test,y,n):
    RFE_cv=2 # Number of cross validations
    estimator=DecisionTreeClassifier()
    FS_RFECV = RFECV(estimator=estimator, min_features_to_select=n, cv=RFE_cv,step=1, verbose=0)
    X_RFECV=FS_RFECV.fit(FS_X, y)
    RFECV_rank = FS_RFECV.ranking_
    X_RFECV_mask= np.empty(len(RFECV_rank), dtype=bool)
    RFECV_rank_with_idx = enumerate(RFECV_rank)
    RFECV_sorted_ranks_idx = sorted(RFECV_rank_with_idx, key=lambda x: x[1])
    RFECV_top_n_idx = [idx for idx, rnk in RFECV_sorted_ranks_idx[:n]]
    for i in range (len(X_RFECV_mask)):
      X_RFECV_mask[i]=False
    for i in range(len(RFECV_top_n_idx)):
      index=RFECV_top_n_idx[i]
      X_RFECV_mask[index] = True
    X_RFECV_df = FS_X.loc[:, X_RFECV_mask]
    X_RFECV_train_df = FS_X_train.loc[:, X_RFECV_mask]
    X_RFECV_test_df = FS_X_test.loc[:, X_RFECV_mask]
    print ('Dataframe with RFECV ready', X_RFECV_df.shape)
    return X_RFECV_df, X_RFECV_train_df, X_RFECV_test_df

def shapRFECV_FS(FS_X,FS_X_train, FS_X_test,y,n):
    print ('Shape of y:', y.shape)
    if (len (y.shape)==2):
      print('Size not 1')
      y=y.iloc[:,0]
    clf = lightgbm.LGBMClassifier(max_depth=5, class_weight='balanced')
    param_grid = {
    'n_estimators': [5, 7, 10],
    'num_leaves': [3, 5, 7, 10],
    }
    search = RandomizedSearchCV(clf, param_grid)
    shapFS_RFECV = ShapRFECV(
    clf=search, step=0.2, cv=2, scoring='roc_auc', n_jobs=3)
    shap_kwargs={'check_additivity':False}
    X_shapRFECV = shapFS_RFECV.fit_compute(FS_X, y, **shap_kwargs)
    X_shapRFECV[['num_features', 'features_set', 'val_metric_mean']]
    performance_plot = shapFS_RFECV.plot()
    print (shapFS_RFECV.get_reduced_features_set(num_features=n))
    selected_fs=shapFS_RFECV.get_reduced_features_set(num_features=n)
    X_shapRFECV_df=FS_X[selected_fs]
    X_shapRFECV_train_df=FS_X_train[selected_fs]
    X_shapRFECV_test_df=FS_X_test[selected_fs]
    print ('Dataframe with shapRFECV ready' , X_shapRFECV_df.shape)
    return X_shapRFECV_df, X_shapRFECV_train_df, X_shapRFECV_test_df

def SModelRanTree(FS_X,FS_X_train, FS_X_test,y,n):
    FS_SModelRanTree=ExtraTreesClassifier(n_estimators=50)
    FS_SModelRanTree.fit(FS_X, y)
    model_Tree = SelectFromModel(FS_SModelRanTree, prefit=True)
    X_Tree=model_Tree.transform(FS_X)
    tree_importance=[]
    count = 0
    for i in FS_X.columns.values:
        tree_importance.insert(count, (i,FS_SModelRanTree.feature_importances_[count]))
        count=count+1
    tree_importance.sort(key=itemgetter(1), reverse=True)
    print ('\n Selected features for tree classifier')
    print ('Dataframe with model selection (random tree):')
    ids=[]
    values=[]
    for i in range( len(tree_importance)):
      id, value = tree_importance[i]
      ids.append(id)
      values.append(value)
    plt.figure(figsize=(20,5))
    plt.plot(ids, values)
    plt.ylabel('Feature importance score')
    plt.xlabel('Feature-ID')
    plt.show()
    res_tree=[]
    for i in range(n):
        res_tree.append(tree_importance[i][0])
    X_SModelRanTree_df=FS_X[res_tree]
    X_SModelRanTree_train_df=FS_X_train[res_tree]
    X_SModelRanTree_test_df=FS_X_test[res_tree]
    return X_SModelRanTree_df, X_SModelRanTree_train_df, X_SModelRanTree_test_df

def SModelLassoCV(FS_X,FS_X_train, FS_X_test,y,n):
    estimator=LassoCV(cv=5)
    iterations=100
    v=False
    FS_SModelLassoCV=SelectFromModel(estimator, threshold=-1000)
    X_LassoCV=FS_SModelLassoCV.fit(FS_X, y)
    if verbose:
      print('Ranking of features', X_LassoCV.n_features_in_)
      print('Rank score of features', X_LassoCV.estimator_.coef_)
    high_score=np.max(X_LassoCV.estimator_.coef_)
    low_score=np.min(X_LassoCV.estimator_.coef_)
    threshold_step=(high_score - low_score)/iterations
    start_threshold=high_score+threshold_step
    if verbose:
      print ('Max/Min/Iterations/step value of coef:',high_score, low_score, iterations,threshold_step)
      print ('Starting value of threshold:',start_threshold)
    FS_SModelLassoCV.threshold=start_threshold
    n_features=FS_SModelLassoCV.transform(FS_X).shape[1]
    i=0
    for i in range (iterations):
      i+=1
      FS_SModelLassoCV.threshold-=threshold_step
      X_transform = FS_SModelLassoCV.transform(FS_X)
      if v:
        print ('X_transform',X_transform )
      n_features = X_transform.shape[1]
      if (n_features>=n):
        break
    X_LassoCV_mask=FS_SModelLassoCV.get_support()
    X_SModelLassoCV_df = FS_X.loc[:, X_LassoCV_mask]
    X_SModelLassoCV_train_df = FS_X_train.loc[:, X_LassoCV_mask]
    X_SModelLassoCV_test_df = FS_X_test.loc[:, X_LassoCV_mask]
    print ('Dataframe with LassoCV (model selection) for n=',n,'best features')
    return X_SModelLassoCV_df, X_SModelLassoCV_train_df, X_SModelLassoCV_test_df


def GenSelect(FS_X,FS_X_train, FS_X_test,y,n):
    FS_XSize=FS_X.shape[1]
    if (FS_XSize<n):
      n=FS_XSize
    estimator = DecisionTreeClassifier()
    max_features= n
    FS_GenSelect=GeneticSelectionCV(estimator, cv=5, verbose=0, scoring="accuracy", max_features=max_features,
    n_population=100, crossover_proba=0.5, mutation_proba=0.2, n_generations=50, crossover_independent_proba=0.5,
    mutation_independent_proba=0.04, tournament_size=3, n_gen_no_change=10, caching=True, n_jobs=-1)
    X_GenSelect=FS_GenSelect.fit_transform(FS_X, y)
    X_GenSelect_mask=FS_GenSelect.get_support()
    X_GenSelect_df = FS_X.loc[:, X_GenSelect_mask]
    X_GenSelect_train_df = FS_X_train.loc[:, X_GenSelect_mask]
    X_GenSelect_test_df = FS_X_test.loc[:, X_GenSelect_mask]
    print ('Dataframe with genetic selection for n=',n,'best features')
    return X_GenSelect_df, X_GenSelect_train_df, X_GenSelect_test_df


def aKDFE_eval_PS (f_selected_in, y ,Normalization, dataset, featuregroup_list, cv_iter=1):
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    '''
    print_border('#', 50)
    print('Number of crossvalidations', cv_iter)
    print ('Dataset to be used:', dataset)
    print ('Used featuregroup:', featuregroup_list)
    print_border('#', 50)
    results = []
    names = []
    Modelrun=0
    local_run_report_df= pd.DataFrame(columns=run_report_col)

    for fg in featuregroup_list:
      for cv in range(1,cv_iter+1):
        X_pd, y_pd,X_train, X_test, y_train, y_test=make_csv_df(dataset_import, y, ';', imp_mean, split, ratio, fg, dataset_type)
        print ('Normalization', Normalization)
        X_train_init=X_train
        X_test_init=X_test
        y_train_init=y_train
        y_test_init= y_test
        X_pd_init = X_pd
        for k in Normalization:
          if (k=='Yes'):
            Norm=k
            print('Normalization of df performed')
            y_train=dfNorm(y_train_init, 'y')
            y_test=dfNorm(y_test_init,'y')
            X_train_norm=dfNorm(X_train_init, 'X')
            X_test_norm=dfNorm(X_test_init, 'X')
            X_pd_norm=dfNorm(X_pd_init, 'X')
          else:
            Norm='No'
            print('No normalization of df performed')
            y_train=y_train_init
            y_test=y_test_init
            X_train_norm=X_train_init
            X_test_norm=X_test_init
            X_pd_norm=X_pd_init
          #########################################
          #Feature selection
          ##########################################
          for FS_name, method in selection_methods:
              if (Norm == 'No' and fg in ('FE-GEN', 'ORGFE') and FS_name =='KBestN'):
                skip_FS_method = True
                print ('Skipping the feature selection method:',FS_name, 'for featuregroup', fg, 'and Normalization', Norm )
                print_border('-',30)
                continue
              print ('Selected FS-method:', FS_name)
              print ('Size of dataset before FS-method',X_pd_norm.shape, X_train_norm.shape, X_test_norm.shape,y_pd.shape )
              if FS_name != 'NO_FS':
                X_pd, X_train, X_test=eval(method+'(X_pd_norm, X_train_norm, X_test_norm,y_pd,n)')
              else:
                 X_pd, X_train, X_test = X_pd_norm, X_train_norm, X_test_norm
              this_selected=pd.DataFrame(sorted(X_train), columns=['feature'])
              this_selected.insert(0,'Dataset',dataset )
              this_selected.insert(1,'Dataset type',dataset_type )
              this_selected.insert(2,'FS_method',method )
              this_selected.insert(3,'Norm',Norm )
              this_selected.insert(4,'Featuregroup',fg )
              f_selected_in.append(this_selected)
              f_selected_out= pd.concat(f_selected_in, ignore_index=True)
              if verbose:
                print ('Selected features df:',f_selected_out )
              print ('Norm before modeling', Norm)
              y_train_arr=np.array(y_train).flatten()
              y_test_arr=np.array(y_test).flatten()
              for i in (X_train, y_train, X_test, y_test, y_train_arr, y_test_arr):
                  print ('Shape of X_train, y_train, X_test, y_test, y_train_arr, y_test_arr:', i.shape)
              for name, model in models:
                  Modelrun+=1
                  print('\n')
                  print_border('#', 50)
                  print('Modelrun#',Modelrun,', Starting:',time_diff(starttime, time.time()),', Dataset:', dataset, ', Datseset type:',dataset_type, 'Featuregroup:',fg,', CV#',cv,', Normalization:', Norm,', FS-Method:', FS_name,', Class-Model:', name,'-' ,model)
                  clf = model.fit(X_train, y_train_arr)
                  y_pred_arr = clf.predict(X_test)
                  y_pred = pd.DataFrame(y_pred_arr, columns = ['Y_hat'])
                  print('Modelrun#',Modelrun,', Finished:', time_diff(starttime, time.time()),'Featuregroup:',fg,', CV#',cv,', Normalization:', Norm,', FS-Method:', FS_name,', Class-Model:',name  )
                  ######################################################################
                  #Calculation of ROCAUC for predicted y (y-hat) and accuracy score
                  class_rocauc=roc_auc_score(y_test_arr, y_pred_arr)
                  test_accur_score=model.score(X_test, y_test_arr)
                  if verbose:
                    print ('Run#',cv,', Classified ROC_AUC score on (y_test, y_pred):', cv,', for model ',name, ':', class_rocauc,
                          ', Norm (Yes/No):', Norm, ', FS-Method: ', FS_name, ', on dataset:', dataset)
                  print_border('-', 50)
                  ####################################################################
                  if dataset_type == 'RAW':
                      #Calculation of ROCAUC on majority vote on outcome (Y) group by PID (X)
                      #Setting same index on dataframes for join
                      X_test_length = len(X_test)
                      test_index=pd.Index(range(0, X_test_length, 1))
                      X_test=X_test.set_index(test_index)
                      y_test=y_test.set_index(test_index)
                      result_df=pd.concat([X_test, y_test, y_pred], axis=1, join="inner")
                      result_df=add_y_major_vote(result_df, 'PID', 'Y1_hat')
                      #Result for unique patients
                      result_UnPID_df=make_PID_unique_df(result_df)
                      #Arrays for ROC_AUC-value
                      y_test_arr_mv=np.array(result_UnPID_df['Y1']).flatten()
                      y_pred_arr_mv=np.array(result_UnPID_df['Y_mv']).flatten()
                      print('Shape of test/predicted arrays for unique PID (roc_auc_mv) ', y_test_arr_mv.shape, y_pred_arr_mv.shape)
                      class_rocauc_mv=roc_auc_score(y_test_arr_mv, y_pred_arr_mv)
                  if dataset_type=='TIDY':
                      class_rocauc_mv = 0
                      X_test_length = len(X_test)
                      test_index=pd.Index(range(0, X_test_length, 1))
                      X_test=X_test.set_index(test_index)
                      y_test=y_test.set_index(test_index)
                      result_df=pd.concat([X_test, y_test, y_pred], axis=1, join="inner")

                      ####################################################################
                  #Adding calculations to result-df
                  local_run_report_df.loc[len(local_run_report_df)] = [Modelrun, cv, dataset, dataset_type ,Norm, FS_name, fg, name, test_accur_score, class_rocauc, class_rocauc_mv]
                  if reporting:
                    print('local run report:')
                    display (local_run_report_df)
                  if verbose:
                    print('Shape and info of result dataframe:', result_df.shape, result_df.columns.values)
                    print_border('-', 30)
                    for i in (X_test, y_test, y_pred, result_df):
                      print ('Shape ... of X_test, y_test, y_pred, result_pd:', i.shape, i.head(1))

    return f_selected_out, y_pred, result_df, local_run_report_df


In [ ]:

 #Import files from Drive
!cp "/content/drive/MyDrive/PHD/S14/dataset/S14_TOTAL_DATASET_V2.csv" "S14_TOTAL_DATASET_V2.csv"

dataset='S14_TOTAL_DATASET_V2'
dataset_import=dataset+'.csv'
if dataset=='S14_TOTAL_DATASET_V4':
  dataset_type='RAW'
elif dataset=='S14_TOTAL_DATASET_V2':
    dataset_type='TIDY'
print('Dataset type', dataset_type)

#Import locally on computer
#dataset_import='C:/Users/olbjaa/Documents/PHD/Dataset/'+dataset_import
#dataset='C:/Users/olbjaa/Documents/PHD/Dataset/'+dataset_import

#dataset_import_test = "/content/drive/My Drive/PHD/S14/Results/"
print('Dataset_import name:', dataset_import)

org_list = ["10", "30", "10-E", "20-E", "30-E", "10-M", "20-M", "30-M", "10-S", "20-S", "30-S", "10-W", "20-W", "30-W"]
idea_list = ["320","350","400", "180-E", "180-M", "180-S", "180-W", "215-E", "215-M", "215-S", "215-W", "320-E", "320-M", "320-S", "320-W", "350-E", "350-M", "350-S", "350-W", "400-E", "400-M", "400-S", "400-W"]
y=['Y1']

#Init values
#featuregroup_list = ['ORG', 'IDEATION', 'FE-GEN', 'ORGIDEA', 'ORGFE']
featuregroup_list = ['FE-GEN', 'ORGIDEA', 'ORGFE']

result_df = []
f_selected = []

imp_mean=1
split=1
ratio=0.3
#Numbers of features to select
n=5
reporting = False
verbose=False
cv_iter = 2
Normalization=['Yes', 'No']
#Normalization=['No']
print ('Dataset to be used:', dataset)
print ('Used featuregroup:', featuregroup_list)
print ('Number of cv:', cv_iter)
print ('Used normalization:', Normalization)

print_border('-')
selection_methods = [('NO_FS', 'No Feature selection'),
('VT', 'FSVarianceThreshold'),
('KBestN','KBestN' ),
('KBestMut', 'KBestMutual'),
('RFE', 'RFE_FS'),
('RFECV', 'RFECV_FS'),
('shapRFECV_FS', 'shapRFECV_FS'),
('GenSelect', 'GenSelect'),
('SModTree','SModelRanTree'),
('SModLasso', 'SModelLassoCV')]

#selection_methods = [('VT', 'FSVarianceThreshold'), ('KBestN','KBestN')]

print('Total feature selection methods to be used:', len(selection_methods))
for j in range (len(selection_methods)):
  print (selection_methods[j][0])

models = [( 'LogReg', LogisticRegression() ),
('RF', RandomForestClassifier()),
('KNN', KNeighborsClassifier()),
#('SVM', SVC()),
('GNB', GaussianNB()),
('XGB', XGBClassifier()),
('ANN-Small',MLPClassifier(hidden_layer_sizes=(15,10,5))),
('ANN-Large',MLPClassifier(hidden_layer_sizes=(150,100,50)))
]

#models = [( 'LogReg', LogisticRegression() )]
print_border('-')
print('Total classification models to be used:', len(models))
for j in range (len(models)):
  print (models[j][0])


In [ ]:
from sklearn.metrics.pairwise import normalize

########################################################################
#Run experiment

print ('USED GPU:', tf.test.gpu_device_name())
run_report_col=['Model#','CV#','Dataset','Dataset type','Normalizations','FS_method','Featuregroups','Class_modell','Test_accuracy', 'Test_ROC_AUC','Test_ROC_AUC_MV' ]
total_run_report_df=pd.DataFrame(columns=run_report_col)


starttime=time.time()
#starttime=time_diff(0,0)
print('Execution started at:',time_diff(0,starttime))

print('\n')
print_border('#', 50)
exp_selected_f, y_pred, result_df, local_run_report_df =aKDFE_eval_PS(f_selected,y ,Normalization,  dataset,featuregroup_list, cv_iter)
#print ('Result after classification for FS-method:',FS, '-- Normalization:',Norm, '-- Dataset:',dataset, end = "")
#display (exp_result)
print ('stop4')
total_run_report_df=total_run_report_df.append(local_run_report_df)
display (total_run_report_df)
display (len(y_pred))
print_border('#', 50)
#print ('Result after feature selection:',FS, '-- Normalization:',Norm, '-- Dataset:',dataset, end = "")
display (exp_selected_f)
print_border('#', 50)
print('Number of selected feature by FS-method')
selected_grouped = exp_selected_f.groupby([ 'FS_method', 'Norm']).size()
print (selected_grouped)
filename_class = dataset+'_'+dataset_type+'_ROCAUC_score_'+actual_time()+'.csv'
filename_FS = dataset+'_'+dataset_type+'_selected_features_'+actual_time()+'.csv'
local_run_report_df.to_csv(filename_class)
exp_selected_f.to_csv(filename_FS)
print (filename_class, filename_FS)
!cp {filename_class} "/content/drive/MyDrive/PHD/S14/Results/V5"
!cp {filename_FS} "/content/drive/MyDrive/PHD/S14/Results/V5"
files.download(filename_class)
files.download(filename_FS)



In [ ]:
#Summary of results and presenting
#import pandas as pd
pd.set_option('display.max_rows', 100)
dataset="/content/drive/My Drive/PHD/S10/Results/S1_AED_Fracture_v0_fe_total_selected_features.csv"
S10_total_results = pd.read_csv(dataset,sep=',')
dataset="/content/drive/My Drive/PHD/S10/Results/S1_AED_Fracture_v0_org_total_selected_features.csv"
S10_total_temp = pd.read_csv(dataset,sep=',')
S10_total_results=S10_total_results.append(S10_total_temp)
dataset="/content/drive/My Drive/PHD/S10/Results/HBA1c_v1_fe_total_selected_features.csv"
S10_total_temp = pd.read_csv(dataset,sep=',')
S10_total_results=S10_total_results.append(S10_total_temp)
dataset="/content/drive/My Drive/PHD/S10/Results/HBA1c_v1_org_total_selected_features.csv"
S10_total_temp = pd.read_csv(dataset,sep=',')
S10_total_results=S10_total_results.append(S10_total_temp)
print ('Size before feature typing', S10_total_results.shape)
var_typing(S10_total_results)

print ('Size efter feature typing', S10_total_results.shape)

print (S10_total_results.groupby(['dataset', 'FS_method', 'norm']).size())

selected_f_summary=S10_total_results[['dataset', 'f_code']].groupby(['dataset',"f_code"]).size().reset_index()
selected_f_summary.sort_values(by=['dataset', 'f_code'])
print (selected_f_summary )
display(selected_f_summary.plot(legend=True, figsize=(12,4)))